In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.3 MB/s eta 0:00:00


In [ ]:
from Bio import Entrez, SeqIO
import re

In [ ]:
import re

def extract_gene_info(info_string):
    # Normalize the string: remove extra spaces and newlines
    info_string = ' '.join(info_string.split())

    # Extract gene ID
    gene_id_match = re.search(r'ID:\s*(\d+)', info_string)
    gene_id = gene_id_match.group(1) if gene_id_match else None

    # Extract chromosome number
    chromosome_match = re.search(r'Chromosome:\s*(\d+)', info_string)
    chromosome = chromosome_match.group(1) if chromosome_match else None

    # Extract scaffold, start, end, and complement information
    annotation_match = re.search(r'Annotation:\s*([\w\.]+)\s*\((\d+)\.\.(\d+)(,\s*complement)?\)', info_string)
    if annotation_match:
        scaffold = annotation_match.group(1)
        start = int(annotation_match.group(2))
        end = int(annotation_match.group(3))
        is_complement = annotation_match.group(4) is not None
    else:
        scaffold, start, end, is_complement = None, None, None, None

    # Extract gene name
    gene_name_match = re.match(r'\d+\.\s*(\w+)', info_string)
    gene_name = gene_name_match.group(1) if gene_name_match else None

    # Extract organism name
    organism_match = re.search(r'\[(.*?)\]', info_string)
    organism = organism_match.group(1) if organism_match else None

    return {
        'gene_id': gene_id,
        'chromosome': chromosome,
        'scaffold': scaffold,
        'start': start,
        'end': end,
        'is_complement': is_complement,
        'gene_name': gene_name,
        'organism': organism
    }


def get_info_string(gene_id):
    handle = Entrez.efetch(db="gene", id=gene_id, rettype="fasta", retmode="text")
    fasta_sequence = handle.read()
    handle.close()

    return fasta_sequence


def pull_genome(gene_info):
  handle = Entrez.efetch(db="nuccore",
                       id=gene_info['scaffold'],
                       seq_start=gene_info['start'],
                       seq_stop=gene_info['end'],
                       strand=2 if gene_info['is_complement'] else 1,
                       rettype="fasta",
                       retmode="text")

  record = SeqIO.read(handle, "fasta")
  handle.close()

  record.description = f"{gene_info['scaffold']}:{'c' if gene_info['is_complement'] else ''}{gene_info['end']}-{gene_info['start']} {gene_info['gene_name']} [organism={gene_info['organism']}] [GeneID={gene_info['gene_id']}] [chromosome={gene_info['chromosome']}]"

  return {
      'sequence': f"{record.seq}",
      'description': f">{record.description}"
  }


def format_row(id:str):
  id = str(id)
  try:
    info_string = get_info_string(id)
    info_string_beta = info_string[1:].replace('\n', ' ')
    gene_info = extract_gene_info(info_string_beta)

    return pull_genome(gene_info)
    # return gene_info
  except Exception as e:
    print(f"Error with {id}: {e}")
    return {
        'sequence': None,
        'description': None
    }

In [ ]:
Entrez.email = "grand.hunter.dark.14@gmail.com"

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('OneOmics_CHO_Global_Spectral_Library_with_gene_ids_filled3.csv')
df.head(10)

,log_prec_y,Gene ID
0,0.485237,100682525
1,0.258538,100682526
2,0.632336,100682527
3,0.993341,100682528
4,0.541653,100682529
5,0.136222,100682531
6,0.480341,100682532
7,0.598669,100682533
8,0.437047,100682534
9,0.642578,100682536


In [ ]:
format_row("100682525")

{'sequence': 'AAGCCTGGCTGACTTCTTGGGTGCTGCCATGGAGGAGCCACAGTCAGACCTCAGCATCGAGCTCCCTCTGAGCCAGGAGACATTTTCAGACCTGTGGAAACTGTGAGTGCATCTTATGGGGCCCCTAAGGTATACCCCTCCATGCATCCTCCCCCAATCCTAGCACCCTGTCCGATAGCAGGGGCTTATGAGAAGCAAAAACAGTAGTTAGGTGTCTGTGATCCCTGTATTTGGGAGGTGTGAAGGCAGGGGGATGCAAAGTTTGAGATCATCCTCAACTACATAGCAAGTTCCAGGTCAGCCTGGCGTAAGGGAGATCCTGTGTACAAAAATGGAAATCAGTAGTGCTAACTCTGCTCTCTACTCTTGTCTTGTTTTACAGACTTCCTCCAAACAATGTTCTGGTAAGGACCAGGGCAGAAAGGGACTGAGGGTTTGCTGGGCTGCTCTGGTGGGATACTGAGAACACAGCATTGAGGTCCTTCTTTGGTCCCTTCACAGTCCACCTTGCCGTCATCCGATTCCATTGAAGAGCTGTTCCTGTCAGAGAATGTTACAGGCTGGTTAGAAGACTCAGGTGGAGCGCTCCAAGGGGTGGCAGCAGCAGCAGCTTCTACAGCAGAAGACCCTGTAACTGAGACTCCTGCACCCCTGGCCTCTGCACCAGCCACTCCCTGGCCCCTCTCATCTTCTGTCCCATCCTATAAAACCTACCAGGGTGACTATGGCTTCCGTCTGGGCTTCCTGCACTCAGGGACGGCCAAATCTGTCACGTGCACGGTCAGTGTCCGTGAGGTGTTGACTTTCATGTGACCTTTCAATATCTGTTGTTTTCTGTCTAGATTTTGGGGGTTCCTCTTTAGCCCATGGACTTTGATGTATGTCTCAGAGTTTAATTTCCCCCTGAACCTTTGCCTTTTATCCTTCTCTCCTATTCTCTGCATCTCTCCAGGGGACATGGAACTTTCTTTCCTCACATTCCTTTC

In [ ]:
import pandas as pd
from tqdm import tqdm

def process_csv(input_path, output_path):
    # Read the CSV file
    print("Reading CSV file...")
    df = pd.read_csv(input_path)
    # df = df.head(50)

    # Check if required columns exist
    if 'Gene ID' not in df.columns or 'Gene ID' not in df.columns:
        raise ValueError("Input CSV must have 'gene' and 'expression' columns")

    print("Processing genes...")
    # Apply the format_row function to each gene and create new columns
    tqdm.pandas(desc="Processing")
    new_columns = df['Gene ID'].progress_apply(format_row).apply(pd.Series)
    df['description'] = new_columns['description']
    df['sequence'] = new_columns['sequence']

    print("Writing updated CSV...")
    # Write the updated dataframe to a new CSV file
    df.to_csv(output_path, index=False)
    print(f"Updated CSV saved to {output_path}")

# Example usage
input_csv_path = 'OneOmics_CHO_Global_Spectral_Library_with_gene_ids_filled3.csv'
output_csv_path = 'output.csv'

process_csv(input_csv_path, output_csv_path)

Reading CSV file...
Processing genes...


Processing:   0%|          | 4/11066 [00:02<2:06:54,  1.45it/s]

Error with 100682527: HTTP Error 400: Bad Request


Processing:   4%|▎         | 397/11066 [06:52<3:04:00,  1.03s/it]

Error with 113832273: HTTP Error 400: Bad Request


...

Processing: 100%|██████████| 11066/11066 [3:14:39<00:00,  1.25it/s]

Error with 113839106: HTTP Error 400: Bad Request


Processing: 100%|██████████| 11066/11066 [3:14:40<00:00,  1.06s/it]

Error with 113839127: HTTP Error 400: Bad Request


Writing updated CSV...
Updated CSV saved to output.csv


In [ ]:
input_csv_path = 'final_sum_nozero_log.csv'
output_csv_path = 'output.csv'

df = pd.read_csv(input_csv_path)
df.head(10)

,gene,expression
0,100760238,0.148916
1,100760532,0.387830
2,100761113,0.384858
3,100760824,0.762337
4,100754742,0.047732
...,...,...
137,100770670,0.851191
138,103164539,0.016339
139,100771533,0.477842
140,100772106,0.256273


In [ ]:
df = pd.read_csv('output.csv')

df.head(78)

,log_prec_y,Gene ID,description,sequence
0,0.485237,100682525,>NW_003613643.1:1643801-1638391 Tp53 [organism...,AAGCCTGGCTGACTTCTTGGGTGCTGCCATGGAGGAGCCACAGTCA...
1,0.258538,100682526,>NW_003619211.1:c16599-9011 Tuba1c [organism=C...,AGTCTATATAAGACCTCTCCTGGAGGCGGACTCACACTGCTTCTCC...
2,0.632336,100682527,NaN,NaN
3,0.993341,100682528,>NW_003614949.1:69399-49056 Tuba1b [organism=C...,CTGTATATAAGGAGCCGCATCCGCCCATGTGCCGCAGGTTCTCTTA...
4,0.541653,100682529,>NW_003614027.1:1194116-1180637 Mgat1 [organis...,TTGCAGGGGATATCTGTGGAAGGTGCCAGCATCTAGGTCTGGGTAG...
...,...,...,...,...
73,0.684353,100689238,>NW_003618735.1:14841-8051 Aimp1 [organism=Cri...,AGTTTTACAGGCAACTTTGAGAGAAGAAAAGAAACTTCGAGTTGAA...
74,0.275077,100689239,>NW_003614541.1:c254164-205442 Dicer1 [organis...,GTTACCTAGGGTATGCATAAATACAGACTCGGAAACTCTGAAAGAA...
75,0.596491,100689240,>NW_003614751.1:c151747-54150 Glg1 [organism=C...,GGGATGTTTCGCTTGTCGGCGGCGTTGCAGCTGCTGCTGCTAGCGG...
76,0.619086,107976996,>NW_003613822.1:630603-594779 Por [organism=Cr...,GGATAGGGTTTCCCTGTGTTTCCCTGGCTATCCTGGAACTTGCTCT...


In [ ]:
df.to_csv('output1.csv', index=False)

In [ ]:
def process_csv_2(input_path, output_path, id_list):
    # Read the CSV file
    print("Reading CSV file...")
    df = pd.read_csv(input_path)

    # Check if required columns exist
    if 'gene' not in df.columns or 'expression' not in df.columns or 'description' not in df.columns:
        raise ValueError("Input CSV must have 'gene' and 'expression' columns")

    # Filter the dataframe to include only the specified IDs
    df_to_process = df[df['gene'].isin([int(x) for x in id_list])]

    print(f"Processing {len(df_to_process)} genes...")
    # Apply the format_row function to each gene and create new columns
    tqdm.pandas(desc="Processing")
    new_columns = df_to_process['gene'].progress_apply(format_row).apply(pd.Series)

    print(new_columns)

    # Update the main dataframe with the new values
    df.loc[df['gene'].isin([int(x) for x in id_list]), 'description'] = new_columns['description']
    df.loc[df['gene'].isin([int(x) for x in id_list]), 'sequence'] = new_columns['sequence']

    print("Writing updated CSV...")
    # Write the updated dataframe to a new CSV file
    df.to_csv(output_path, index=False)
    print(f"Updated CSV saved to {output_path}")

# Example usage
input_csv_path = 'output.csv'
output_csv_path = 'output_all.csv'
ids_to_process =  [
"100768761", "100760933", "100760828", "100752670", "103159893",
"103159909", "100769155", "100759273", "100759197", "100755226",
"100772698", "100751411", "103164149", "100768478", "103164217",
"103164300", "103163234", "100820693", "100763453", "103159970",
"103160246", "103160622", "100767931", "103162679", "103161735",
"103162768", "100774848", "100761150", "100759983", "103162747",
"100763190", "100750963", "100689230", "100757208", "103158571",
"100763665", "100764248", "100689241", "100766101", "100773121",
"100773413", "100759420", "100755663", "103163326", "100752132",
"103163945", "100752427", "103159483", "100761655", "100773419",
"100767951", "100756249", "100771105", "100761274", "103161498",
"103162936", "100770548", "103160096", "103160742", "100764663",
"103163212", "103161818", "103162614", "100766807", "100753222",
"100771300", "103161134", "103164089", "100775059", "100752627",
"100689088", "103161509", "103159537", "103160450", "100771500",
"103159272", "103159855", "103161236", "100757719", "103161120",
"100753536", "100759534", "103159188", "100758296", "103161548",
"100752453", "100772867", "100771131", "103160268", "100767877",
"103164191", "100757531", "103160875", "103160417", "103162840",
"103160886", "100757915", "103162620", "100754039", "100750509",
"100767596", "103164241", "100761591", "100754044", "100765276",
"100753362", "103161555", "100751101", "103159477", "103162096",
"103161627", "100767038", "100751602", "103159922", "100774336",
"103163253", "100773179", "100757745", "103159937", "100751900",
"100767803", "100758327", "100758905", "100773277", "103161097",
"103161096", "103159497", "103161599", "103160139", "100759383",
"100757842", "103159916", "100771460", "100773864", "103164543",
"100768890", "100764520", "100769258", "103160053", "100765396",
"100768314", "100773966", "100774944", "100750466", "100774654",
"103158807", "103158883", "103158959", "103158984", "103158990",
"100760765", "103159000", "103159022", "100774851", "103159056"
]

process_csv_2(input_csv_path, output_csv_path, ids_to_process)

Reading CSV file...
Processing 155 genes...


Processing:   3%|▎         | 4/155 [00:02<01:19,  1.90it/s]

Error with 100760828


Processing:   3%|▎         | 5/155 [00:02<01:30,  1.66it/s]

Error with 100752670


...

Error with 103163234


Processing:  12%|█▏        | 19/155 [00:13<01:25,  1.60it/s]

Error with 100820693


Processing:  13%|█▎        | 20/155 [00:14<01:29,  1.51it/s]

Error with 100763453
